# Практическая часть 4
Создание предсказаний, оценка моделей и ранжирование признаков

## Блок 0. Подготовка окружения

Импортировать библиотеки для работы с данными, моделями машинного обучения и метриками качества.
Настроить отображение таблиц.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.feature_selection import mutual_info_classif

%matplotlib inline

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

## Блок 1. Подготовка данных для задачи классификации

### 1.1. Загрузка набора Iris и формирование признаков/цели

Загрузить набор данных Iris.
Добавить человекочитаемое название вида.
Разделить данные на матрицу признаков `X` и целевую переменную `y`.

In [ ]:
iris = datasets.load_iris(as_frame=True)
df_iris = iris.frame.copy()
df_iris["species"] = df_iris["target"].map(dict(enumerate(iris.target_names)))

X = df_iris[iris.feature_names]
y = df_iris["species"]

df_iris.head()

### 1.2. Разделение на обучающую и тестовую выборки

Разбить данные на обучающую и тестовую выборки в пропорции 70/30.
Использовать стратификацию по целевой переменной.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape

## Блок 2. Базовая модель и качество на тестовой выборке

### 2.1. Обучение логистической регрессии

Обучить модель логистической регрессии на обучающей выборке.
Оценить точность (accuracy) на тестовой выборке.

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_log = log_reg.predict(X_test)
acc_log = accuracy_score(y_test, y_pred_log)
acc_log

### 2.2. Отчёт по качеству модели

Получить более подробный отчёт:
- матрицу ошибок (confusion matrix);
- текстовый `classification_report`.

In [ ]:
cm_log = confusion_matrix(y_test, y_pred_log, labels=iris.target_names)
cm_log

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_log, display_labels=iris.target_names)
disp.plot()
plt.title("Confusion matrix — Logistic Regression")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred_log))

## Блок 3. Сравнение нескольких моделей с кросс-валидацией

### 3.1. Определение набора моделей

Определить несколько базовых моделей для сравнения:
- логистическая регрессия;
- решающее дерево;
- k-ближайших соседей.

In [ ]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

### 3.2. Кросс-валидация (accuracy) для каждой модели

Для каждой модели провести кросс-валидацию (например, 5-кратную) по метрике accuracy.
Свести результаты в таблицу со средним значением и стандартным отклонением.

In [ ]:
cv_results = []

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring="accuracy")
    cv_results.append({
        "model": name,
        "mean_accuracy": scores.mean(),
        "std_accuracy": scores.std()
    })

cv_results_df = pd.DataFrame(cv_results)
cv_results_df

### 3.3. Выбор лучшей модели по кросс-валидации

По таблице кросс-валидации выбрать модель с наилучшим средним значением accuracy.
С этой моделью продолжить дальнейший анализ.

In [ ]:
best_model_name = cv_results_df.sort_values("mean_accuracy", ascending=False).iloc[0]["model"]
best_model_name

## Блок 4. Создание предсказаний для новых объектов

### 4.1. Обучение выбранной модели на всех обучающих данных

Взять выбранную модель (например, решающее дерево).
Обучить её на обучающей выборке и получить предсказания на тестовой.

In [ ]:
best_model = models["DecisionTree"]

best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

accuracy_score(y_test, y_pred_best)

### 4.2. Пример предсказаний для первых нескольких объектов

Показать, какие предсказания модель даёт для первых 10 объектов тестовой выборки.
Сравнить реальные и предсказанные виды.

In [ ]:
pred_df = pd.DataFrame({
    "true_species": y_test.values,
    "predicted_species": y_pred_best
}, index=y_test.index)

pred_df.head(10)

## Блок 5. Ранжирование признаков (оценка важности)

### 5.1. Важность признаков по модели решающего дерева

Использовать обученную модель решающего дерева для оценки важности признаков (`feature_importances_`).
Сформировать таблицу с важностями и отсортировать признаки по убыванию важности.

In [ ]:
importances = best_model.feature_importances_

feat_importance_df = pd.DataFrame({
    "feature": iris.feature_names,
    "tree_importance": importances
}).sort_values("tree_importance", ascending=False)

feat_importance_df

### 5.2. Коэффициенты взаимной информации (mutual information)

Дополнительно оценить значимость признаков с помощью взаимной информации `mutual_info_classif`.
Добавить значения в общую таблицу ранжирования.

In [ ]:
mi_scores = mutual_info_classif(X, y, random_state=42)

feat_importance_df = feat_importance_df.merge(
    pd.DataFrame({
        "feature": iris.feature_names,
        "mutual_info": mi_scores
    }),
    on="feature"
).sort_values("tree_importance", ascending=False)

feat_importance_df

### 5.3. Визуализация важности признаков

Построить горизонтальную столбчатую диаграмму важностей признаков по решающему дереву.

In [ ]:
plt.figure(figsize=(6, 4))
plt.barh(feat_importance_df["feature"], feat_importance_df["tree_importance"])
plt.gca().invert_yaxis()
plt.xlabel("Важность (DecisionTree)")
plt.title("Ранжирование признаков для задачи классификации Iris")
plt.tight_layout()
plt.show()

## Блок 6. Текстовые выводы

В отдельных markdown-ячейках сделать выводы по результатам работы:
- какая модель показала наилучшее качество по кросс-валидации;
- насколько хорошо она предсказывает виды на тестовой выборке (accuracy, матрица ошибок);
- какие признаки оказываются наиболее важными;
- насколько получившиеся классы интерпретируемы с точки зрения предметной области.